In [1]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
# config_proto.gpu_options.allow_growth = True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

Using TensorFlow backend.


In [2]:
X_train = np.load('../../20_Galaxy/ANGRY/2HXT-train.npy')
Y_train = np.load('../../20_Galaxy/ANGRY/2HXT-train_label.npy')

X_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val.npy')
Y_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val_label.npy')

In [3]:
one_hot = preprocessing.OneHotEncoder(sparse = False)
y_train = one_hot.fit_transform(Y_train)
y_valid = one_hot.fit_transform(Y_valid)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
model.add(Attention_layer())

model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Tensor("attention_layer_3/div:0", shape=(?, 169, 32), dtype=float32)
Tensor("lstm_3/transpose_1:0", shape=(?, ?, 32), dtype=float32)
Tensor("attention_layer_3/mul:0", shape=(?, 169, 32), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 174s 11ms/step - loss: 0.6915 - acc: 0.5277 - val_loss: 0.6919 - val_acc: 0.5085
Epoch 2/50
15658/15658 [==============================] - 167s 11ms/step - loss: 0.6664 - acc: 0.5752 - val_loss: 0.6130 - val_acc: 0.6810
Epoch 3/50
15658/15658 [==============================] - 168s 11ms/step - loss: 0.5974 - acc: 0.6704 - val_loss: 0.5664 - val_acc: 0.7670
Epoch 4/50
15658/15658 [==============================] - 169s 11ms/step - loss: 0.5222 - acc: 0.7369 - val_loss: 0.5384 - val_acc: 0.8015
Epoch 5/50
15658/15658 [==============================] - 170s 11ms/step - loss: 0.4806 - acc: 0.7698 - val_loss: 0.5172 - val_acc: 0.8263
Epoch 6/50
15658/15658 [==============================

In [8]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

# model.add(LSTM(32, return_sequences=True,
#                batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 17s 1ms/step - loss: 0.6223 - acc: 0.6369 - val_loss: 0.5352 - val_acc: 0.8115
Epoch 2/50
15658/15658 [==============================] - 15s 975us/step - loss: 0.5028 - acc: 0.7608 - val_loss: 0.5246 - val_acc: 0.8615
Epoch 3/50
15658/15658 [==============================] - 15s 964us/step - loss: 0.4616 - acc: 0.7839 - val_loss: 0.5110 - val_acc: 0.8490
Epoch 4/50
15658/15658 [==============================] - 15s 966us/step - loss: 0.4479 - acc: 0.7934 - val_loss: 0.5128 - val_acc: 0.8480
Epoch 5/50
15658/15658 [==============================] - 16s 1ms/step - loss: 0.4324 - acc: 0.8032 - val_loss: 0.4536 - val_acc: 0.8635
Epoch 6/50
15658/15658 [==============================] - 16s 992us/step - loss: 0.4127 - acc: 0.8159 - val_loss: 0.5623 - val_acc: 0.8315
Epoch 7/50
15658/15658 [==============================] - 16s 993us/step - loss: 0.4098 - acc: 0.8209 - val_loss: 0.4376 

In [9]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=True,
               batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(32, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 167s 11ms/step - loss: 0.5899 - acc: 0.6572 - val_loss: 0.7344 - val_acc: 0.7270
Epoch 2/50
15658/15658 [==============================] - 165s 11ms/step - loss: 0.4692 - acc: 0.7783 - val_loss: 0.5785 - val_acc: 0.8207
Epoch 3/50
15658/15658 [==============================] - 164s 10ms/step - loss: 0.4364 - acc: 0.7993 - val_loss: 0.5373 - val_acc: 0.8642
Epoch 4/50
15658/15658 [==============================] - 163s 10ms/step - loss: 0.4078 - acc: 0.8170 - val_loss: 0.4429 - val_acc: 0.8685
Epoch 5/50
15658/15658 [==============================] - 163s 10ms/step - loss: 0.3916 - acc: 0.8242 - val_loss: 0.4277 - val_acc: 0.8778
Epoch 6/50
15658/15658 [==============================] - 163s 10ms/step - loss: 0.3763 - acc: 0.8354 - val_loss: 0.2913 - val_acc: 0.8955
Epoch 7/50
15658/15658 [==============================] - 163s 10ms/step - loss: 0.3700 - acc: 0.8377 - val_loss: 0.3

In [16]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 32
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(LSTM(64, return_sequences=True,
               batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 171s 11ms/step - loss: 0.5849 - acc: 0.6740 - val_loss: 0.4300 - val_acc: 0.8402
Epoch 2/50
15658/15658 [==============================] - 170s 11ms/step - loss: 0.4720 - acc: 0.7771 - val_loss: 0.4378 - val_acc: 0.8400
Epoch 3/50
15658/15658 [==============================] - 171s 11ms/step - loss: 0.4386 - acc: 0.7989 - val_loss: 0.4555 - val_acc: 0.8748
Epoch 4/50
15658/15658 [==============================] - 169s 11ms/step - loss: 0.4243 - acc: 0.8101 - val_loss: 0.4210 - val_acc: 0.8520
Epoch 5/50
15658/15658 [==============================] - 168s 11ms/step - loss: 0.4155 - acc: 0.8138 - val_loss: 0.4082 - val_acc: 0.8765
Epoch 6/50
15658/15658 [==============================] - 168s 11ms/step - loss: 0.3946 - acc: 0.8218 - val_loss: 0.4116 - val_acc: 0.8820
Epoch 7/50
15658/15658 [==============================] - 167s 11ms/step - loss: 0.3828 - acc: 0.8309 - val_loss: 0.3

In [25]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 64
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
# model.add(LSTM(64, return_sequences=True,
#                batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 164s 10ms/step - loss: 0.6527 - acc: 0.5926 - val_loss: 0.5284 - val_acc: 0.7395
Epoch 2/50
15658/15658 [==============================] - 148s 9ms/step - loss: 0.5232 - acc: 0.7337 - val_loss: 0.4876 - val_acc: 0.8300
Epoch 3/50
15658/15658 [==============================] - 146s 9ms/step - loss: 0.4883 - acc: 0.7598 - val_loss: 0.4972 - val_acc: 0.8432
Epoch 4/50
15658/15658 [==============================] - 146s 9ms/step - loss: 0.4668 - acc: 0.7779 - val_loss: 0.5028 - val_acc: 0.8552
Epoch 5/50
15658/15658 [==============================] - 147s 9ms/step - loss: 0.4465 - acc: 0.7904 - val_loss: 0.5435 - val_acc: 0.8625
Epoch 6/50
15658/15658 [==============================] - 149s 9ms/step - loss: 0.4366 - acc: 0.7982 - val_loss: 0.5185 - val_acc: 0.8660
Epoch 7/50
15658/15658 [==============================] - 151s 10ms/step - loss: 0.4252 - acc: 0.8038 - val_loss: 0.4872 -

In [4]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 64
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
# model.add(LSTM(64, return_sequences=True,
#                batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

/usr/local/lib/python2.7/dist-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 166s 11ms/step - loss: 0.6481 - acc: 0.6021 - val_loss: 0.5342 - val_acc: 0.7360
Epoch 2/50
15658/15658 [==============================] - 159s 10ms/step - loss: 0.5531 - acc: 0.7091 - val_loss: 0.4947 - val_acc: 0.8297
Epoch 3/50
15658/15658 [==============================] - 157s 10ms/step - loss: 0.4907 - acc: 0.7572 - val_loss: 0.4781 - val_acc: 0.8425
Epoch 4/50
15658/15658 [==============================] - 157s 10ms/step - loss: 0.4515 - acc: 0.7841 - val_loss: 0.5140 - val_acc: 0.8678
Epoch 5/50
15658/15658 [==============================] - 162s 10ms/step - loss: 0.4293 - acc: 0.8023 - val_loss: 0.5263 - val_acc: 0.8737
Epoch 6/50
15658/15658 [==============================] - 153s 10ms/step - loss: 0.4170 - acc: 0.8095 - val_loss: 0.5568 - val_acc: 0.8792
Epoch 7/50
15658/15658 [==============================] - 149s 10ms/step - loss: 0.4008 - acc: 0.8169 - val_loss: 0.5

In [8]:
class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
batch_size = 64
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2), input_shape=(12* 3640,1)))
model.add(Convolution1D(6, (21), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(7)))
model.add(Dropout(0.25))

model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(7, (13), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(Convolution1D(5, (9), kernel_initializer='glorot_uniform', activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2)))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Dropout(0.25))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
# model.add(LSTM(64, return_sequences=True,
#                batch_input_shape=(batch_size, 12, 3640)))
# model.add(LSTM(32, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(Attention_layer())
model.add(Flatten())
model.add(Dense(2, kernel_initializer='glorot_uniform', activation='softmax'))

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)

model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])
# score = model.evaluate(x_test, y_test, batch_size=batch_size)
# print model.predict(x_test)
# print K.eval(categorical_accuracy(y_test, model.predict(x_test)))
# print score

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 152s 10ms/step - loss: 0.6429 - acc: 0.6068 - val_loss: 0.5259 - val_acc: 0.7468
Epoch 2/50
15658/15658 [==============================] - 148s 9ms/step - loss: 0.5485 - acc: 0.7116 - val_loss: 0.4551 - val_acc: 0.8393
Epoch 3/50
15658/15658 [==============================] - 150s 10ms/step - loss: 0.4960 - acc: 0.7542 - val_loss: 0.4584 - val_acc: 0.8488
Epoch 4/50
15658/15658 [==============================] - 158s 10ms/step - loss: 0.4730 - acc: 0.7746 - val_loss: 0.4395 - val_acc: 0.8638
Epoch 5/50
15658/15658 [==============================] - 158s 10ms/step - loss: 0.4525 - acc: 0.7863 - val_loss: 0.4533 - val_acc: 0.8423
Epoch 6/50
15658/15658 [==============================] - 151s 10ms/step - loss: 0.4400 - acc: 0.7944 - val_loss: 0.4487 - val_acc: 0.8695
Epoch 7/50
15658/15658 [==============================] - 146s 9ms/step - loss: 0.4320 - acc: 0.8040 - val_loss: 0.450

In [10]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    print drop3
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("dropout_21/cond/Merge:0", shape=(?, 169, 5), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.6266 - acc: 0.6305 - val_loss: 0.5893 - val_acc: 0.7827
Epoch 2/50
15658/15658 [==============================] - 15s 974us/step - loss: 0.5381 - acc: 0.7295 - val_loss: 0.5640 - val_acc: 0.8275
Epoch 3/50
15658/15658 [==============================] - 15s 958us/step - loss: 0.4917 - acc: 0.7694 - val_loss: 0.5613 - val_acc: 0.8430
Epoch 4/50
15658/15658 [==============================] - 15s 977us/step - loss: 0.4656 - acc: 0.7894 - val_loss: 0.6243 - val_acc: 0.8678
Epoch 5/50
15658/15658 [==============================] - 15s 969us/step - loss: 0.4486 - acc: 0.7940 - val_loss: 0.5309 - val_acc: 0.8518
Epoch 6/50
15658/15658 [==============================] - 15s 973us/step - loss: 0.4352 - acc: 0.8046 - val_loss: 0.5768 - val_acc: 0.8755
Epoch 7/50
15658/15658 [===========================

In [17]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    print drop3
    
    lstm_out1 = LSTM(32,return_sequences=True)(drop3)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("dropout_42/cond/Merge:0", shape=(?, 169, 5), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 178s 11ms/step - loss: 0.6271 - acc: 0.6282 - val_loss: 0.5044 - val_acc: 0.8123
Epoch 2/50
15658/15658 [==============================] - 174s 11ms/step - loss: 0.4888 - acc: 0.7627 - val_loss: 0.5399 - val_acc: 0.8608
Epoch 3/50
15658/15658 [==============================] - 172s 11ms/step - loss: 0.4453 - acc: 0.7925 - val_loss: 0.5881 - val_acc: 0.8670
Epoch 4/50
15658/15658 [==============================] - 170s 11ms/step - loss: 0.4186 - acc: 0.8112 - val_loss: 0.6356 - val_acc: 0.8742
Epoch 5/50
15658/15658 [==============================] - 169s 11ms/step - loss: 0.3924 - acc: 0.8272 - val_loss: 0.5285 - val_acc: 0.8855
Epoch 6/50
15658/15658 [==============================] - 177s 11ms/step - loss: 0.3783 - acc: 0.8332 - val_loss: 0.5341 - val_acc: 0.8713
Epoch 7/50
15658/15658 [=========================

In [18]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(32,return_sequences=True)(concat)

    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_1/concat:0", shape=(?, 10, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 57s 4ms/step - loss: 0.5976 - acc: 0.6595 - val_loss: 0.4791 - val_acc: 0.8030
Epoch 2/50
15658/15658 [==============================] - 46s 3ms/step - loss: 0.4805 - acc: 0.7668 - val_loss: 0.4611 - val_acc: 0.8495
Epoch 3/50
15658/15658 [==============================] - 47s 3ms/step - loss: 0.4515 - acc: 0.7860 - val_loss: 0.6569 - val_acc: 0.7642
Epoch 4/50
15658/15658 [==============================] - 48s 3ms/step - loss: 0.4332 - acc: 0.7972 - val_loss: 0.5046 - val_acc: 0.8713
Epoch 5/50
15658/15658 [==============================] - 48s 3ms/step - loss: 0.4166 - acc: 0.8081 - val_loss: 0.4316 - val_acc: 0.8668
Epoch 6/50
15658/15658 [==============================] - 48s 3ms/step - loss: 0.3983 - acc: 0.8188 - val_loss: 0.4176 - val_acc: 0.8892
Epoch 7/50
15658/15658 [==============================] - 48s 

In [30]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_13/concat:0", shape=(?, 120, 40), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 232s 15ms/step - loss: 0.6566 - acc: 0.5912 - val_loss: 0.5443 - val_acc: 0.7390
Epoch 2/50
15658/15658 [==============================] - 201s 13ms/step - loss: 0.5279 - acc: 0.7320 - val_loss: 0.4743 - val_acc: 0.8005
Epoch 3/50
15658/15658 [==============================] - 200s 13ms/step - loss: 0.4795 - acc: 0.7679 - val_loss: 0.5262 - val_acc: 0.8200
Epoch 4/50
15658/15658 [==============================] - 198s 13ms/step - loss: 0.4543 - acc: 0.7866 - val_loss: 0.4723 - val_acc: 0.8592
Epoch 5/50
15658/15658 [==============================] - 200s 13ms/step - loss: 0.4408 - acc: 0.7959 - val_loss: 0.4633 - val_acc: 0.8605
Epoch 6/50
15658/15658 [==============================] - 201s 13ms/step - loss: 0.4255 - acc: 0.8036 - val_loss: 0.4941 - val_acc: 0.8645
Epoch 7/50
15658/15658 [========================

In [31]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_14/concat:0", shape=(?, 10, 480), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 127s 8ms/step - loss: 0.6484 - acc: 0.6037 - val_loss: 0.5259 - val_acc: 0.7775
Epoch 2/50
15658/15658 [==============================] - 101s 6ms/step - loss: 0.5181 - acc: 0.7448 - val_loss: 0.4817 - val_acc: 0.8063
Epoch 3/50
15658/15658 [==============================] - 101s 6ms/step - loss: 0.4713 - acc: 0.7756 - val_loss: 0.4874 - val_acc: 0.8423
Epoch 4/50
15658/15658 [==============================] - 101s 6ms/step - loss: 0.4482 - acc: 0.7897 - val_loss: 0.4920 - val_acc: 0.8645
Epoch 5/50
15658/15658 [==============================] - 101s 6ms/step - loss: 0.4277 - acc: 0.8048 - val_loss: 0.5027 - val_acc: 0.8637
Epoch 6/50
15658/15658 [==============================] - 103s 7ms/step - loss: 0.4075 - acc: 0.8149 - val_loss: 0.4928 - val_acc: 0.8810
Epoch 7/50
15658/15658 [==============================

In [32]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(48, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(56, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(40, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
#     lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(concat)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_15/concat:0", shape=(?, 10, 480), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 116s 7ms/step - loss: 0.6242 - acc: 0.6399 - val_loss: 0.5417 - val_acc: 0.7445
Epoch 2/50
15658/15658 [==============================] - 91s 6ms/step - loss: 0.5138 - acc: 0.7471 - val_loss: 0.5240 - val_acc: 0.8393
Epoch 3/50
15658/15658 [==============================] - 92s 6ms/step - loss: 0.4751 - acc: 0.7728 - val_loss: 0.5293 - val_acc: 0.8520
Epoch 4/50
15658/15658 [==============================] - 92s 6ms/step - loss: 0.4543 - acc: 0.7898 - val_loss: 0.5672 - val_acc: 0.8570
Epoch 5/50
15658/15658 [==============================] - 92s 6ms/step - loss: 0.4364 - acc: 0.8044 - val_loss: 0.5922 - val_acc: 0.8652
Epoch 6/50
15658/15658 [==============================] - 92s 6ms/step - loss: 0.4226 - acc: 0.8054 - val_loss: 0.5741 - val_acc: 0.8680
Epoch 7/50
15658/15658 [==============================] - 9

In [48]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
#     lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(concat)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=20, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_31/concat:0", shape=(?, 120, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 130s 8ms/step - loss: 0.6787 - acc: 0.5531 - val_loss: 0.6589 - val_acc: 0.6522
Epoch 2/50
15658/15658 [==============================] - 77s 5ms/step - loss: 0.6514 - acc: 0.6175 - val_loss: 0.6161 - val_acc: 0.6723
Epoch 3/50
15658/15658 [==============================] - 78s 5ms/step - loss: 0.6281 - acc: 0.6450 - val_loss: 0.5773 - val_acc: 0.7145
Epoch 4/50
15658/15658 [==============================] - 79s 5ms/step - loss: 0.5889 - acc: 0.6923 - val_loss: 0.5170 - val_acc: 0.7997
Epoch 5/50
15658/15658 [==============================] - 79s 5ms/step - loss: 0.5402 - acc: 0.7318 - val_loss: 0.4760 - val_acc: 0.8223
Epoch 6/50
15658/15658 [==============================] - 79s 5ms/step - loss: 0.5133 - acc: 0.7489 - val_loss: 0.4899 - val_acc: 0.7897
Epoch 7/50
15658/15658 [==============================] - 8

In [49]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
#     lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
    flatten1 = Attention_layer()(concat)

#     flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_32/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("attention_layer_3/div:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_32/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("attention_layer_3/mul:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 200s 13ms/step - loss: 0.6708 - acc: 0.5710 - val_loss: 0.6338 - val_acc: 0.6585
Epoch 2/50
15658/15658 [==============================] - 138s 9ms/step - loss: 0.5684 - acc: 0.7036 - val_loss: 0.5869 - val_acc: 0.7825
Epoch 3/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4973 - acc: 0.7627 - val_loss: 0.4947 - val_acc: 0.8452
Epoch 4/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4659 - acc: 0.7844 - val_loss: 0.4530 - val_acc: 0.8645
Epoch 5/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4379 - acc: 0.8031 - val_loss: 0.4156 - 

In [50]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
#     lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
    flatten1 = Attention_layer()(concat)

#     flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_33/concat:0", shape=(?, 338, 30), dtype=float32)
Tensor("attention_layer_4/div:0", shape=(?, 338, 30), dtype=float32)
Tensor("concatenate_33/concat:0", shape=(?, 338, 30), dtype=float32)
Tensor("attention_layer_4/mul:0", shape=(?, 338, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 202s 13ms/step - loss: 0.6837 - acc: 0.5393 - val_loss: 0.6644 - val_acc: 0.5677
Epoch 2/50
15658/15658 [==============================] - 136s 9ms/step - loss: 0.6173 - acc: 0.6519 - val_loss: 0.6293 - val_acc: 0.7847
Epoch 3/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.5246 - acc: 0.7375 - val_loss: 0.5577 - val_acc: 0.8435
Epoch 4/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4849 - acc: 0.7680 - val_loss: 0.6092 - val_acc: 0.8250
Epoch 5/50
15658/15658 [==============================] - 140s 9ms/step - loss: 0.4655 - acc: 0.7815 - val_loss: 0.5383 - 

In [53]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=100,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_36/concat:0", shape=(?, 169, 90), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/100
15658/15658 [==============================] - 487s 31ms/step - loss: 0.6546 - acc: 0.5865 - val_loss: 0.5624 - val_acc: 0.7165
Epoch 2/100
15658/15658 [==============================] - 391s 25ms/step - loss: 0.5358 - acc: 0.7228 - val_loss: 0.5172 - val_acc: 0.7935
Epoch 3/100
15658/15658 [==============================] - 392s 25ms/step - loss: 0.4809 - acc: 0.7664 - val_loss: 0.5177 - val_acc: 0.8465
Epoch 4/100
15658/15658 [==============================] - 412s 26ms/step - loss: 0.4574 - acc: 0.7817 - val_loss: 0.4864 - val_acc: 0.8500
Epoch 5/100
15658/15658 [==============================] - 410s 26ms/step - loss: 0.4435 - acc: 0.7923 - val_loss: 0.4946 - val_acc: 0.8508
Epoch 6/100
15658/15658 [==============================] - 412s 26ms/step - loss: 0.4314 - acc: 0.8000 - val_loss: 0.5062 - val_acc: 0.8698
Epoch 7/100
15658/15658 [=================

In [54]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=100,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_37/concat:0", shape=(?, 507, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/100
15658/15658 [==============================] - 851s 54ms/step - loss: 0.6653 - acc: 0.5771 - val_loss: 0.6032 - val_acc: 0.6937
Epoch 2/100
15658/15658 [==============================] - 719s 46ms/step - loss: 0.5802 - acc: 0.6841 - val_loss: 0.4751 - val_acc: 0.8110
Epoch 3/100
15658/15658 [==============================] - 725s 46ms/step - loss: 0.5030 - acc: 0.7467 - val_loss: 0.4760 - val_acc: 0.8205
Epoch 4/100
15658/15658 [==============================] - 719s 46ms/step - loss: 0.4681 - acc: 0.7739 - val_loss: 0.4599 - val_acc: 0.8630
Epoch 5/100
15658/15658 [==============================] - 726s 46ms/step - loss: 0.4421 - acc: 0.7947 - val_loss: 0.4731 - val_acc: 0.8670
Epoch 6/100
15658/15658 [==============================] - 717s 46ms/step - loss: 0.4275 - acc: 0.8003 - val_loss: 0.4803 - val_acc: 0.8598
Epoch 7/100
15658/15658 [=================

In [57]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = Bidirectional(LSTM(64,return_sequences=True))(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=20, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=100,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_40/concat:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/100
15658/15658 [==============================] - 545s 35ms/step - loss: 0.6520 - acc: 0.5991 - val_loss: 0.5544 - val_acc: 0.7518
Epoch 2/100
15658/15658 [==============================] - 468s 30ms/step - loss: 0.5237 - acc: 0.7349 - val_loss: 0.5231 - val_acc: 0.7768
Epoch 3/100
15658/15658 [==============================] - 468s 30ms/step - loss: 0.4720 - acc: 0.7716 - val_loss: 0.4618 - val_acc: 0.8595
Epoch 4/100
15658/15658 [==============================] - 465s 30ms/step - loss: 0.4405 - acc: 0.7940 - val_loss: 0.4651 - val_acc: 0.8445
Epoch 5/100
15658/15658 [==============================] - 466s 30ms/step - loss: 0.4201 - acc: 0.8055 - val_loss: 0.4449 - val_acc: 0.8793
Epoch 6/100
15658/15658 [==============================] - 468s 30ms/step - loss: 0.3967 - acc: 0.8249 - val_loss: 0.4498 - val_acc: 0.8660
Epoch 7/100
15658/15658 [=================